## Zillow Housing--Feature EDA

There's already some excellent kernel in python, you can have a reference.    

[Missing Values & Multicollinearity--vivek][1]  
  
[Simple Exploration Notebook - SRK][2]

**my object**  
 To have easy and quick roadmap to what i have done to playing the data.

  [1]: https://www.kaggle.com/viveksrinivasan/zillow-eda-on-missing-values-multicollinearity
  [2]: https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-zillow-prize

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import pandas as pd
import numpy as np
import seaborn as sns
import missingno as msno
color = sns.color_palette()
from matplotlib import pyplot as plt
pd.set_option("display.max_columns",100)
plt.style.use("ggplot")

# Reading In Dataset

In [ ]:
## read in the property data and merge with trainId
train = pd.read_csv('../input/train_2016.csv', parse_dates=["transactiondate"])
properties = pd.read_csv('../input/properties_2016.csv')
merged = pd.merge(train,properties,on="parcelid",how="left")
del properties
merged.head()

# Shape and size

In [ ]:
## find no error predict
merged_no_err = merged[merged.logerror ==0]
merged_err = merged[merged.logerror !=0]

## calculate rows
total_row = merged.shape[0]
no_err_row = merged_no_err.shape[0]
err_row = merged_err.shape[0]
print("total row is: {:d}".format(total_row))
print("no error row is: {:d}".format(no_err_row))
print("error row is: {:d}".format(err_row))

# missing data analysis  

---  
From the beginning of EDA , we should know what's happening in our missing data? Is there some logic problem? Or some doable cleaning and wrangling. We should not abandon the missing feature or samples immediately.  
For example, the missing correlation map, we can using a dict mapping to fill the na when the cor <1. 

In [ ]:
## select the column with missing value
missingValueColumns = merged.columns[merged.isnull().any()].tolist()

## plot missing value
(merged[missingValueColumns].isnull().sum()/total_row).sort_values().plot.barh(figsize=(10,16))
msno.heatmap(merged[missingValueColumns],figsize=(20,20))

# Object anaylsis

Do some modification of some outlier, we can have a look at the beautiful distribution of our logerror.

In [ ]:
ulimit = np.percentile(merged.logerror.values, 99)
llimit = np.percentile(merged.logerror.values, 1)
merged['logerror'].ix[merged['logerror']>ulimit] = ulimit
merged['logerror'].ix[merged['logerror']<llimit] = llimit

plt.figure(figsize=(12,8))
sns.distplot(merged.logerror.values, bins=50, kde=False)
plt.xlabel('logerror', fontsize=12)
plt.show()

From the fig, we see about 30k samples contribute 80% error, so may be we can focus on samples have bad prediction.
And our threshold is [-inf, -0.05] U [0.05, inf].  
Maybe in the future we can analysis the model from postive error and negative error.

In [ ]:
## calculate cdf 
(abs(merged_err.logerror).sort_values().cumsum()/abs(merged_err.logerror).sum())\
.sort_values(ascending=False).reset_index(drop=True).plot.line(title="cdf error of feature")

## choose the 80% error of samplews
per8_merged = merged_err[\
                         ((abs(merged_err.logerror).sort_values().cumsum()\
                           /abs(merged_err.logerror).sum())\
                          >0.2).sort_index()]

## split the data
pos_err = per8_merged[per8_merged.logerror > 0]
neg_err = per8_merged[per8_merged.logerror < 0]

print("The threshold is [-inf,-0.05] | [0.05,inf]")

In [ ]:
var

In [ ]:
col_name = merged.columns.tolist()
col_name = col_name[:5]
for var in col_name:
    merged[var].value_counts().sort_index().plot.bar()

### That's just the begining.  Stay tunes. 